<a href="https://colab.research.google.com/github/CatarinaRRF/ChurnOfClients-AluraVoz/blob/main/data_analises/churn_clients_initial_treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Tratamento dos dados**

## *Dados Importados (API - Alura Voz)*

*   Quais são as informações inuteis?
*   O que eu quero fazer com esses dados?


---



In [ ]:
import pandas as pd

In [ ]:
data_alura_voz = pd.read_json ('Telco-Customer-Churn.json')

In [ ]:
data_alura_voz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


In [ ]:
data_alura_voz.dtypes

customerID    object
Churn         object
customer      object
phone         object
internet      object
account       object
dtype: object

In [ ]:
data_alura_voz.columns = ['ClienteId', 'Churn', 'Informações do cliente', 'Serviço de celular', 'Serviço de internet', 'Especificações da Conta']
data_alura_voz.columns
data_alura_voz.head (3)

,ClienteId,Churn,Informações do cliente,Serviço de celular,Serviço de internet,Especificações da Conta
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [ ]:
client_id = data_alura_voz['ClienteId'];
churn = data_alura_voz['Churn'];

###*Verificando dados inconsistentes do churn e ClientId* 

In [ ]:
data_alura_voz['Churn'].unique ()

array(['No', 'Yes', ''], dtype=object)

In [ ]:
data_alura_voz['ClienteId'].unique ().shape[0]

7267

##*Extraindo informações apartir dos clientes*

---



In [ ]:
info_cliente = pd.json_normalize(data=data_alura_voz['Informações do cliente'])
info_cliente = info_cliente.join([client_id, churn]);
info_cliente.head(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,ClienteId,Churn
0,Female,0,Yes,Yes,9,0002-ORFBO,No
1,Male,0,No,No,9,0003-MKNFE,No
2,Male,0,No,No,4,0004-TLHLJ,Yes
3,Male,1,Yes,No,13,0011-IGKFF,Yes
4,Female,1,Yes,No,3,0013-EXCHZ,Yes
5,Female,0,No,Yes,9,0013-MHZWF,No
6,Female,1,Yes,No,71,0013-SMEOE,No
7,Male,0,Yes,No,63,0014-BMAQU,No
8,Female,1,No,No,7,0015-UOCOJ,No
9,Female,0,Yes,Yes,65,0016-QLJIS,No


In [ ]:
info_cliente = info_cliente.reindex(columns=['ClienteId','SeniorCitizen','Partner','Dependents','tenure', 'gender','Churn'])
info_cliente.columns = ['ClienteId','Cidadão Senior', 'Parceiros', 'Dependentes', 'ternure', 'Sexo', 'Churn']
info_cliente.head(10)

,ClienteId,Cidadão Senior,Parceiros,Dependentes,ternure,Sexo,Churn
0,0002-ORFBO,0,Yes,Yes,9,Female,No
1,0003-MKNFE,0,No,No,9,Male,No
2,0004-TLHLJ,0,No,No,4,Male,Yes
3,0011-IGKFF,1,Yes,No,13,Male,Yes
4,0013-EXCHZ,1,Yes,No,3,Female,Yes
5,0013-MHZWF,0,No,Yes,9,Female,No
6,0013-SMEOE,1,Yes,No,71,Female,No
7,0014-BMAQU,0,Yes,No,63,Male,No
8,0015-UOCOJ,1,No,No,7,Female,No
9,0016-QLJIS,0,Yes,Yes,65,Female,No


####*Verificando dados inconsistentes*


> Inconsistencias no Tenure: '0' é um dado confuso



In [ ]:
info_cliente['Cidadão Senior'].unique ()

array([0, 1])

In [ ]:
info_cliente['Parceiros'].unique ()

array(['Yes', 'No'], dtype=object)

In [ ]:
info_cliente['Dependentes'].unique ()

array(['Yes', 'No'], dtype=object)

In [ ]:
#Meses de contrato do cliente
info_cliente['ternure'].unique ()

array(['Yes', 'No'], dtype=object)

In [ ]:
info_cliente['Sexo'].unique ()

array(['Female', 'Male'], dtype=object)

##*Extraindo informações dos produtos ofertados*
*   Olhar qual serviço tem mais churn
*  Os serviços são: Linhas de Celular, Serviço de Internet e Serviço de Streaming
* Talvez concatenar
* Identificar as infos desecessarias
---



### **Assinatura de Serviço Telefônico**

In [ ]:
servico_celular = pd.json_normalize(data=data_alura_voz['Serviço de celular'])
servico_celular = servico_celular.join([client_id, churn]);
servico_celular.head ()

,PhoneService,MultipleLines,ClienteId,Churn
0,Yes,No,0002-ORFBO,No
1,Yes,Yes,0003-MKNFE,No
2,Yes,No,0004-TLHLJ,Yes
3,Yes,No,0011-IGKFF,Yes
4,Yes,No,0013-EXCHZ,Yes


In [ ]:
servico_celular = servico_celular.reindex(columns=['ClienteId','PhoneService','MultipleLines','Churn'])
servico_celular.columns = ['ClienteId','ServiçoTelefônico', 'MultiplasLinhas?', 'Churn']
servico_celular.head ()

,ClienteId,ServiçoTelefônico,MultiplasLinhas?,Churn
0,0002-ORFBO,Yes,No,No
1,0003-MKNFE,Yes,Yes,No
2,0004-TLHLJ,Yes,No,Yes
3,0011-IGKFF,Yes,No,Yes
4,0013-EXCHZ,Yes,No,Yes


####*Verificando dados inconsistentes*


> Não tem



In [ ]:
servico_celular['ServiçoTelefônico'].unique ()

array(['Yes', 'No'], dtype=object)

In [ ]:
servico_celular['MultiplasLinhas?'].unique ()

array(['No', 'Yes', 'No phone service'], dtype=object)

### **Assinatura do Provedor de Internet (DSL ou Fiber optica)**

In [ ]:
servico_internet = pd.json_normalize(data=data_alura_voz['Serviço de internet'])
servico_internet = servico_internet.join([client_id, churn]);
servico_internet.head ()

,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,ClienteId,Churn
0,DSL,No,Yes,No,Yes,Yes,No,0002-ORFBO,No
1,DSL,No,No,No,No,No,Yes,0003-MKNFE,No
2,Fiber optic,No,No,Yes,No,No,No,0004-TLHLJ,Yes
3,Fiber optic,No,Yes,Yes,No,Yes,Yes,0011-IGKFF,Yes
4,Fiber optic,No,No,No,Yes,Yes,No,0013-EXCHZ,Yes


In [ ]:
#indica somente o serviço relacionadas a internet sem o serviço de streming que vem na string original
servico_provedor = servico_internet.reindex(columns=['ClienteId','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection', 'TechSupport','StreamingTV', 'StreamingMovies', 'Churn'])
servico_provedor.drop (columns=['StreamingTV', 'StreamingMovies'], inplace=True)
servico_provedor.head ()

,ClienteId,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,Churn
0,0002-ORFBO,DSL,No,Yes,No,Yes,No
1,0003-MKNFE,DSL,No,No,No,No,No
2,0004-TLHLJ,Fiber optic,No,No,Yes,No,Yes
3,0011-IGKFF,Fiber optic,No,Yes,Yes,No,Yes
4,0013-EXCHZ,Fiber optic,No,No,No,Yes,Yes


####*Verificando dados inconsistentes*

In [ ]:
servico_provedor['InternetService'].unique ()

array(['DSL', 'Fiber optic', 'No'], dtype=object)

In [ ]:
servico_internet['OnlineSecurity'].unique ()

array(['No', 'Yes', 'No internet service'], dtype=object)

In [ ]:
servico_internet['OnlineBackup'].unique ()

array(['Yes', 'No', 'No internet service'], dtype=object)

In [ ]:
servico_internet['DeviceProtection'].unique ()

array(['No', 'Yes', 'No internet service'], dtype=object)

In [ ]:
servico_internet['TechSupport'].unique ()

array(['Yes', 'No', 'No internet service'], dtype=object)

### **Assinatura Streaming (TV a cabo e streaming de filmes)**

In [ ]:
servico_streaming = pd.DataFrame(data=servico_internet, columns=['StreamingTV','StreamingMovies'])
servico_streaming = servico_streaming.join([client_id, churn]);
servico_streaming.head ()

,StreamingTV,StreamingMovies,ClienteId,Churn
0,Yes,No,0002-ORFBO,No
1,No,Yes,0003-MKNFE,No
2,No,No,0004-TLHLJ,Yes
3,Yes,Yes,0011-IGKFF,Yes
4,Yes,No,0013-EXCHZ,Yes


In [ ]:
servico_streaming = servico_streaming.reindex(columns=['ClienteId','StreamingTV','StreamingMovies','Churn'])
servico_streaming.columns = ['ClienteId', 'StreamingTV', 'StreamingdeFilmes','Churn']
servico_streaming.head ()

,ClienteId,StreamingTV,StreamingdeFilmes,Churn
0,0002-ORFBO,Yes,No,No
1,0003-MKNFE,No,Yes,No
2,0004-TLHLJ,No,No,Yes
3,0011-IGKFF,Yes,Yes,Yes
4,0013-EXCHZ,Yes,No,Yes


####*Verificando dados inconsistentes*

> Tv a cabo mostra opção 'Sem serviço de internet' maia Tv precisa estar vinculada a NET?
> Mesma coisa para os seviços de filme.


In [ ]:
servico_streaming['StreamingTV'].unique ()

array(['Yes', 'No', 'No internet service'], dtype=object)

In [ ]:
servico_streaming['StreamingdeFilmes'].unique ()

array(['No', 'Yes', 'No internet service'], dtype=object)

##*Extraindo informações das contas*

---



In [ ]:
conta = pd.json_normalize(data=data_alura_voz['Especificações da Conta'])
conta = conta.join([client_id, churn]);
conta.head ()

,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,ClienteId,Churn
0,One year,Yes,Mailed check,65.6,593.3,0002-ORFBO,No
1,Month-to-month,No,Mailed check,59.9,542.4,0003-MKNFE,No
2,Month-to-month,Yes,Electronic check,73.9,280.85,0004-TLHLJ,Yes
3,Month-to-month,Yes,Electronic check,98.0,1237.85,0011-IGKFF,Yes
4,Month-to-month,Yes,Mailed check,83.9,267.4,0013-EXCHZ,Yes


In [ ]:
conta = conta.reindex(columns=['ClienteId','Contract','PaperlessBilling','PaymentMethod','Charges.Monthly','Charges.Total','Churn'])
conta.columns = ['ClienteId','Contrato','CobrançaSemPapel?', 'MetodoDePagamento', 'CobrançaMensal','CobrançaTotal','Churn']
conta.head ()

,ClienteId,Contrato,CobrançaSemPapel?,MetodoDePagamento,CobrançaMensal,CobrançaTotal,Churn
0,0002-ORFBO,One year,Yes,Mailed check,65.6,593.3,No
1,0003-MKNFE,Month-to-month,No,Mailed check,59.9,542.4,No
2,0004-TLHLJ,Month-to-month,Yes,Electronic check,73.9,280.85,Yes
3,0011-IGKFF,Month-to-month,Yes,Electronic check,98.0,1237.85,Yes
4,0013-EXCHZ,Month-to-month,Yes,Mailed check,83.9,267.4,Yes


###*Verificando dados inconsistentes*

In [ ]:
conta['Contrato'].unique ()

array(['One year', 'Month-to-month', 'Two year'], dtype=object)

In [ ]:
conta['CobrançaSemPapel?'].unique ()

array(['Yes', 'No'], dtype=object)

In [ ]:
conta['MetodoDePagamento'].unique ()

array(['Mailed check', 'Electronic check', 'Credit card (automatic)',
       'Bank transfer (automatic)'], dtype=object)

In [ ]:
conta['CobrançaMensal'].unique ().shape[0]

1585

In [ ]:
conta['CobrançaTotal'].unique ().shape[0]

6531